tgb - 4/15/2020
- Adapting Ankitesh's notebook that builds and train a "brute-force" network to David Walling's hyperparameter search  
- Adding the option to choose between aquaplanet and real-geography data

In [1]:
import sys
sys.path.insert(1,"/home1/07064/tg863631/anaconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages") #work around for h5py
from cbrain.imports import *
from cbrain.cam_constants import *
from cbrain.utils import *
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
from cbrain.climate_invariant import *

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow_probability as tfp
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
# import cartopy.crs as ccrs
import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
# from climate_invariant import *
from tensorflow.keras import layers
import datetime
from climate_invariant_utils import *
import yaml


/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


## Global Variables

In [2]:
# Load coordinates (just pick any file from the climate model run)

# Comet path below
# coor = xr.open_dataset("/oasis/scratch/comet/ankitesh/temp_project/data/sp8fbp_minus4k.cam2.h1.0000-01-01-00000.nc",\
#                     decode_times=False)

# GP path below
path_0K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/'
coor = xr.open_dataset(path_0K+"AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-09-02-00000.nc")

lat = coor.lat; lon = coor.lon; lev = coor.lev;
coor.close();

In [3]:
# Comet path below
# TRAINDIR = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/CRHData/'
# path = '/home/ankitesh/CBrain_project/CBRAIN-CAM/cbrain/'

# GP path below
TRAINDIR = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path = '/export/nfs0home/tbeucler/CBRAIN-CAM/cbrain/'
path_nnconfig = '/export/nfs0home/tbeucler/CBRAIN-CAM/nn_config/'

# Load hyam and hybm to calculate pressure field in SPCAM
path_hyam = 'hyam_hybm.pkl'
hf = open(path+path_hyam,'rb')
hyam,hybm = pickle.load(hf)

# Scale dictionary to convert the loss to W/m2
scale_dict = load_pickle(path_nnconfig+'scale_dicts/009_Wm2_scaling.pkl')

New Data generator class for the climate-invariant network. Calculates the physical rescalings needed to make the NN climate-invariant

In [4]:
class DataGeneratorClimInv(DataGenerator):
    
    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,                 
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train',portion=1):
        
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        if rh_trans:
            self.qv2rhLayer = QV2RHNumpy(self.inp_sub,self.inp_div,inp_subRH,inp_divRH,hyam,hybm)
        
        if lhflx_trans:
            self.lhflxLayer = LhflxTransNumpy(self.inp_sub,self.inp_div,hyam,hybm)
            
        if t2tns_trans:
            self.t2tnsLayer = T2TmTNSNumpy(self.inp_sub,self.inp_div,inp_subTNS,inp_divTNS,hyam,hybm)
            
        if scaling:
            self.scalingLayer = ScalingNumpy(hyam,hybm)
            self.inp_shape += 1
                    
        if interpolate:
            self.interpLayer = InterpolationNumpy(lev,is_continous,Tnot,lower_lim,interm_size)
            self.inp_shape += interm_size*2 + 4 + 30 ## 4 same as 60-64 and 30 for lev_tilde.size
            
        # tgb - 7/9/2020 - Test only training on a subset of the data determined by portion
        self.portion = portion
            
        
    def __getitem__(self, index):
        
        # If portion<1, only look at a subset of the data by putting an upper bound on index
        if self.portion<1: index = index % round(1/self.portion)
        elif self.portion>1: print('Setting portion=1 because portion>1')
        elif self.portion<0: print('Setting portion=1 because portion<0')
        
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]

        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        X_result = X_norm
        
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result) 
            
        if self.lhflx_trans:
            X_result = self.lhflxLayer.process(X_result)
            X_result = X_result[:,:64]
            X = X[:,:64]
            
        if self.t2tns_trans:
            X_result = self.t2tnsLayer.process(X_result)
        
        if self.scaling:
            scalings = self.scalingLayer.process(X) 
            X_result = np.hstack((X_result,scalings))
        
        if self.interpolate:
            interpolated = self.interpLayer.process(X,X_result)
            X_result = np.hstack((X_result,interpolated))
            

        if self.mode=='val':
            return xr.DataArray(X_result), xr.DataArray(Y)
        return X_result,Y
    
    ##transforms the input data into the required format, take the unnormalized dataset
    def transform(self,X):
        X_norm = self.input_transform.transform(X)
        X_result = X_norm
        
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result)  
        
        if self.lhflx_trans:
            X_result = self.lhflxLayer.process(X_result)
            X_result = X_result[:,:64]
            X = X[:,:64]

        if self.t2tns_trans:
            X_result = self.t2tnsLayer.process(X_result)
        
        if self.scaling:
            scalings = self.scalingLayer.process(X) 
            X_result = np.hstack((X_result,scalings))
        
        if self.interpolate:
            interpolated = self.interpLayer.process(X,X_result)
            X_result = np.hstack((X_result,interpolated))
            

        return X_result

## Data Generators

### Choose between aquaplanet and realistic geography here

In [5]:
# GP paths below
path_aquaplanet = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path_realgeography = ''

# Comet paths below
# path_aquaplanet = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/'
# path_realgeography = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/geography/'

path = path_aquaplanet

### Baseline Data Generator, without ozone as input but with radiative outputs

In [6]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','QRL', 'QRS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_O3 = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE_O3 = '2021_01_24_NORM_O3_small.nc'
VALIDFILE_O3 = '2021_01_24_O3_VALID.nc'
TESTFILE_O3 = '2021_01_24_O3_TEST.nc'

In [7]:
train_gen_noO3 = DataGenerator(
    data_fn = TRAINDIR+TRAINFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [8]:
valid_gen_noO3 = DataGenerator(
    data_fn = TRAINDIR+VALIDFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [9]:
test_gen_noO3 = DataGenerator(
    data_fn = TRAINDIR+TESTFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

### Data Generator using RH

In [10]:
#scale_dict_RH = load_pickle('/home/ankitesh/CBrain_project/CBRAIN-CAM/nn_config/scale_dicts/009_Wm2_scaling_2.pkl')
scale_dict_RH = scale_dict.copy()
scale_dict_RH['RH'] = 0.01*L_S/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%

in_vars_RH = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','QRL','QRS']

# New GP path below
TRAINFILE_RH = '2021_01_24_O3_small_shuffle.nc'
NORMFILE_RH = '2021_02_01_NORM_O3_RH_small.nc'
    
# Comet/Ankitesh path below
# TRAINFILE_RH = 'CI_RH_M4K_NORM_train_shuffle.nc'
# NORMFILE_RH = 'CI_RH_M4K_NORM_norm.nc'
# VALIDFILE_RH = 'CI_RH_M4K_NORM_valid.nc'

In [11]:
train_gen_RH = DataGenerator(
    data_fn = path+TRAINFILE_RH,
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = path+NORMFILE_RH,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict_RH,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using QSATdeficit

We only need the norm file for this generator as we are solely using it as an input to determine the right normalization for the combined generator

In [12]:
# New GP path below
TRAINFILE_QSATdeficit = '2021_02_01_O3_QSATdeficit_small_shuffle.nc'
NORMFILE_QSATdeficit = '2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [13]:
in_vars_QSATdeficit = ['QSATdeficit','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_QSATdeficit = ['PHQ','TPHYSTND','QRL','QRS']

In [14]:
train_gen_QSATdeficit = DataGenerator(
    data_fn = path+TRAINFILE_QSATdeficit,
    input_vars = in_vars_QSATdeficit,
    output_vars = out_vars_QSATdeficit,
    norm_fn = path+NORMFILE_QSATdeficit,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using TNS

In [15]:
in_vars = ['QBP','TfromNS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_TNS = '2021_02_01_O3_TfromNS_small_shuffle.nc'
NORMFILE_TNS = '2021_02_01_NORM_O3_TfromNS_small.nc'
VALIDFILE_TNS = 'CI_TNS_M4K_NORM_valid.nc'

In [16]:
train_gen_TNS = DataGenerator(
    data_fn = path+TRAINFILE_TNS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_TNS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using BCONS

In [17]:
in_vars = ['QBP','BCONS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_BCONS = '2021_02_01_O3_BCONS_small_shuffle.nc'
NORMFILE_BCONS = '2021_02_01_NORM_O3_BCONS_small.nc'

In [18]:
train_gen_BCONS = DataGenerator(
    data_fn = path+TRAINFILE_BCONS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_BCONS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using NSto220

In [19]:
in_vars = ['QBP','T_NSto220','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_T_NSto220 = '2021_03_31_O3_T_NSto220_small.nc'
NORMFILE_T_NSto220 = '2021_03_31_NORM_O3_T_NSto220_small.nc'

In [20]:
train_gen_T_NSto220 = DataGenerator(
    data_fn = path+TRAINFILE_T_NSto220,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_T_NSto220,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using LHF_nsDELQ

In [21]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsDELQ']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_LHF_nsDELQ = '2021_02_01_O3_LHF_nsDELQ_small_shuffle.nc'
NORMFILE_LHF_nsDELQ = '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'

In [22]:
train_gen_LHF_nsDELQ = DataGenerator(
    data_fn = path+TRAINFILE_LHF_nsDELQ,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_LHF_nsDELQ,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using LHF_nsQ

In [23]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsQ']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_LHF_nsQ = '2021_02_01_O3_LHF_nsQ_small_shuffle.nc'
NORMFILE_LHF_nsQ = '2021_02_01_NORM_O3_LHF_nsQ_small.nc'

In [24]:
train_gen_LHF_nsQ = DataGenerator(
    data_fn = path+TRAINFILE_LHF_nsQ,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_LHF_nsQ,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using $O_{3}$

In [25]:
in_vars = ['QBP','TBP','O3_AQUA','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','QRL', 'QRS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_O3 = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE_O3 = '2021_01_24_NORM_O3_small.nc'
VALIDFILE_O3 = '2021_01_24_O3_VALID.nc'
TESTFILE_O3 = '2021_01_24_O3_TEST.nc'

In [26]:
train_gen_O3 = DataGenerator(
    data_fn = TRAINDIR+TRAINFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [27]:
valid_gen_O3 = DataGenerator(
    data_fn = TRAINDIR+VALIDFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

In [28]:
test_gen_O3 = DataGenerator(
    data_fn = TRAINDIR+TESTFILE_O3,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_O3,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True
)

### Data Generator Combined (latest flexible version)

In [29]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
# elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
if path==path_aquaplanet: out_vars=['PHQ','TPHYSTND','QRL','QRS']

In [30]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

TRAINFILE = '2021_03_18_O3_TRAIN_M4K.nc'
#TRAINFILE = '2021_01_24_O3_small_shuffle.nc'
VALIDFILE = '2021_03_18_O3_VALID_M4K.nc'
TESTFILE = '2021_03_18_O3_TEST_P4K.nc'

Old data generator by Ankitesh

In [31]:
# TRAINFILE = 'CI_SP_M4K_train_shuffle.nc'
# NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
# VALIDFILE = 'CI_SP_M4K_valid.nc'


# train_gen_CI = DataGeneratorClimInv(
#     data_fn = path+TRAINFILE,
#     input_vars = in_vars,
#     output_vars = out_vars,
#     norm_fn = path+NORMFILE,
#     input_transform = ('mean', 'maxrs'),
#     output_transform = scale_dict,
#     batch_size=8192,
#     shuffle=True,
#     lev=lev,
#     hyam=hyam,hybm=hybm,
#     inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
#     inp_subTNS=train_gen_RH.input_transform.sub,inp_divTNS=train_gen_RH.input_transform.div,
#     rh_trans=True,t2tns_trans=False,
#     lhflx_trans=False,
#     scaling=False,
#     interpolate=False
# )

# valid_gen_CI = DataGeneratorClimInv(
#     data_fn = path+VALIDFILE,
#     input_vars = in_vars,
#     output_vars = out_vars,
#     norm_fn = path+NORMFILE,
#     input_transform = ('mean', 'maxrs'),
#     output_transform = scale_dict,
#     batch_size=8192,
#     shuffle=True,
#     lev=lev,
#     hyam=hyam,hybm=hybm,
#     inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
#     inp_subTNS=train_gen_RH.input_transform.sub,inp_divTNS=train_gen_RH.input_transform.div,
#     rh_trans=True,t2tns_trans=False,
#     lhflx_trans=False,
#     scaling=False,
#     interpolate=False
# )


Improved flexible data generator

In [55]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling='TfromNS',
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_TNS.input_transform.sub,
                               inp_div_Tscaling=train_gen_TNS.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [56]:
valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling='TfromNS',
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_TNS.input_transform.sub,
                               inp_div_Tscaling=train_gen_TNS.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [57]:
test_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling='TfromNS',
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_TNS.input_transform.sub,
                               inp_div_Tscaling=train_gen_TNS.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

## Add callback class to track loss on multiple sets during training

From [https://stackoverflow.com/questions/47731935/using-multiple-validation-sets-with-keras]

In [58]:
test_gen_CI[0][0].shape

(8192, 64)

In [59]:
np.argwhere(np.isnan(test_gen_CI[0][1]))

array([], shape=(0, 2), dtype=int64)

In [60]:
np.argwhere(np.isnan(test_gen_CI[0][0]))

array([], shape=(0, 2), dtype=int64)

In [61]:
class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            valid_generator,valid_name = validation_set
            #tf.print('Results')
            results = self.model.evaluate_generator(generator=valid_generator)
            #tf.print(results)

            for metric, result in zip(self.model.metrics_names,[results]):
                #tf.print(metric,result)
                valuename = valid_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

## Brute-Force Model

### Climate-invariant (T,Q,PS,S0,SHF,LHF)->($\dot{T}$,$\dot{q}$,RADFLUX)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(64, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
save_name = 'BF_temp'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_pos])

### Ozone (T,Q,$O_{3}$,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(94,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_O3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

### No Ozone (T,Q,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_noO3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
# Nep = 15
# model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
#               callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
              callbacks=[earlyStopping, mcp_save_pos])

### BF linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_18_MLR'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
history.history

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BF NN version with test loss tracking

In [44]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [45]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [47]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_08_NN6L'

In [48]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [49]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [50]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 2542s 441ms/step - loss: 342.0934 - val_loss: 329.9304
Epoch 2/20
5759/5759 [==============================] - 2701s 469ms/step - loss: 320.9061 - val_loss: 316.9567
Epoch 3/20
5759/5759 [==============================] - 2452s 426ms/step - loss: 311.2353 - val_loss: 309.6577
Epoch 4/20
5759/5759 [==============================] - 2254s 391ms/step - loss: 305.5034 - val_loss: 304.9600
Epoch 5/20
5759/5759 [==============================] - 1731s 301ms/step - loss: 301.7379 - val_loss: 301.6836
Epoch 6/20
5759/5759 [==============================] - 1117s 194ms/step - loss: 299.0327 - val_loss: 299.1967
Epoch 7/20
5759/5759 [==============================] - 816s 142ms/step - loss: 296.9427 - val_loss: 297.2431
Epoch 8/20
5759/5759 [==============================] - 713s 124ms/step - loss: 295.2764 - val_loss: 295.7120
Epoch 9/20
5759/5759 [==============================] - 694s 121ms/step - loss: 293.9242 - val_loss: 294.4352
Epoc

In [51]:
history.history

{'loss': [342.09338355590165,
  320.9061308653948,
  311.23525867438974,
  305.5033564990667,
  301.7378943407364,
  299.03272853665186,
  296.9426967053182,
  295.27639429510043,
  293.9242286688753,
  292.8105507327525,
  291.88271119160294,
  291.10301101189106,
  290.43950267433394,
  289.868749243287,
  289.3729577492583,
  288.93778015867997,
  288.5534648097085,
  288.2098106308765,
  287.90122331664634,
  287.6208748516885],
 'val_loss': [329.9303815202876,
  316.9566871866662,
  309.6576747567941,
  304.96000983386693,
  301.6835738202586,
  299.1966588354587,
  297.243100628537,
  295.7119940864864,
  294.43520978061343,
  293.3954472839883,
  292.5363609783128,
  291.8279773312142,
  291.20056871985776,
  290.6817859751116,
  290.2149793161854,
  289.82551079177665,
  289.4662798184087,
  289.1482212480884,
  288.8622394983756,
  288.5948049347219],
 'testP4K_loss': [755.1666537160246,
  749.3020249286134,
  764.911121189238,
  790.1407622900099,
  815.8143627241674,
  838.4

In [52]:
hist_rec = history.history

In [53]:
hist_rec

{'loss': [342.09338355590165,
  320.9061308653948,
  311.23525867438974,
  305.5033564990667,
  301.7378943407364,
  299.03272853665186,
  296.9426967053182,
  295.27639429510043,
  293.9242286688753,
  292.8105507327525,
  291.88271119160294,
  291.10301101189106,
  290.43950267433394,
  289.868749243287,
  289.3729577492583,
  288.93778015867997,
  288.5534648097085,
  288.2098106308765,
  287.90122331664634,
  287.6208748516885],
 'val_loss': [329.9303815202876,
  316.9566871866662,
  309.6576747567941,
  304.96000983386693,
  301.6835738202586,
  299.1966588354587,
  297.243100628537,
  295.7119940864864,
  294.43520978061343,
  293.3954472839883,
  292.5363609783128,
  291.8279773312142,
  291.20056871985776,
  290.6817859751116,
  290.2149793161854,
  289.82551079177665,
  289.4662798184087,
  289.1482212480884,
  288.8622394983756,
  288.5948049347219],
 'testP4K_loss': [755.1666537160246,
  749.3020249286134,
  764.911121189238,
  790.1407622900099,
  815.8143627241674,
  838.4

In [54]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH linear version

In [32]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [35]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH'

In [36]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [37]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [38]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
   3/5759 [..............................] - ETA: 56:44 - loss: 362.5890  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### QSATdeficit linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_QSATdeficit'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### TfromNS linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_TfromNS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BCONS linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_BCONS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Mixed Model

In [24]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [27]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH_BCONS'

In [28]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [29]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [30]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 8098s 1s/step - loss: 324.9880 - val_loss: 313.2678
Epoch 2/10
5759/5759 [==============================] - 12555s 2s/step - loss: 308.3165 - val_loss: 307.0678
Epoch 3/10
5759/5759 [==============================] - 13642s 2s/step - loss: 304.0913 - val_loss: 304.2872
Epoch 4/10
5759/5759 [==============================] - 10465s 2s/step - loss: 301.8938 - val_loss: 302.7330
Epoch 5/10
5759/5759 [==============================] - 7909s 1s/step - loss: 300.5873 - val_loss: 301.7510
Epoch 6/10
5759/5759 [==============================] - 8526s 1s/step - loss: 299.7002 - val_loss: 301.0397
Epoch 7/10
5759/5759 [==============================] - 7230s 1s/step - loss: 299.0066 - val_loss: 300.3937
Epoch 8/10
5759/5759 [==============================] - 8294s 1s/step - loss: 298.4328 - val_loss: 299.8434
Epoch 9/10
5759/5759 [==============================] - 8250s 1s/step - loss: 297.9426 - val_loss: 299.4925
Epoch 10/10
5759/5759 [==

In [31]:
hist_rec = history.history

In [32]:
hist_rec

{'loss': [324.98798001675704,
  308.31648811596017,
  304.0912586973243,
  301.8937511361293,
  300.5873382457078,
  299.7002091802725,
  299.0065637804771,
  298.4328209674489,
  297.9425933249689,
  297.5149222372472],
 'val_loss': [313.2678370875624,
  307.0677993880396,
  304.2871500202989,
  302.73299462630064,
  301.75103527639715,
  301.03966337007205,
  300.39373488234764,
  299.8433922314147,
  299.4924658236212,
  299.0508612894959],
 'testP4K_loss': [696.3224985280676,
  686.2635380876054,
  682.7061903721445,
  680.7840038270242,
  679.5804400619129,
  678.422916433854,
  677.2276842855165,
  676.2437589546028,
  675.5998048438668,
  674.6606088692057]}

In [33]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+(T-TNS)

### RH+NSto220

In [39]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [40]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [42]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_31_MLR_RH_NSto220'

In [43]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [44]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [45]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 10546s 2s/step - loss: 333.2154 - val_loss: 318.1506
Epoch 2/10
5759/5759 [==============================] - 10194s 2s/step - loss: 310.9067 - val_loss: 308.3093
Epoch 3/10
5759/5759 [==============================] - 9862s 2s/step - loss: 304.7300 - val_loss: 304.2289
Epoch 4/10
5759/5759 [==============================] - 11121s 2s/step - loss: 301.8291 - val_loss: 302.1135
Epoch 5/10
5759/5759 [==============================] - 10748s 2s/step - loss: 300.2339 - val_loss: 300.8775
Epoch 6/10
5759/5759 [==============================] - 10117s 2s/step - loss: 299.1923 - val_loss: 299.9938
Epoch 7/10
5759/5759 [==============================] - 10174s 2s/step - loss: 298.3948 - val_loss: 299.3052
Epoch 8/10
5759/5759 [==============================] - 9887s 2s/step - loss: 297.7428 - val_loss: 298.7333
Epoch 9/10
5759/5759 [==============================] - 10299s 2s/step - loss: 297.1934 - val_loss: 298.2455
Epoch 10/10
5759/5759

In [46]:
hist_rec = history.history

In [47]:
hist_rec

{'loss': [333.21542532209264,
  310.90667960419796,
  304.72998016670067,
  301.8290961309116,
  300.2339088743316,
  299.19231390816446,
  298.39477655642935,
  297.74276377828113,
  297.19338681379486,
  296.7217506637878],
 'val_loss': [318.15060213358146,
  308.30926291885646,
  304.22893252623965,
  302.1134895894682,
  300.8775425180225,
  299.9937965616662,
  299.30517368746393,
  298.73325744039545,
  298.2454878341621,
  297.8241368286815],
 'testP4K_loss': [723.0185877851295,
  698.1272752300885,
  685.258197891926,
  679.2784368682783,
  676.55788616959,
  674.8558076801512,
  673.4969380158018,
  672.4355979235329,
  671.423601997825,
  670.7109736936912]}

In [48]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+LHF_nsQ

In [39]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [40]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [42]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH_LHF_nsQ'

In [43]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [44]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [45]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 7449s 1s/step - loss: 332.8352 - val_loss: 318.1170
Epoch 2/10
5759/5759 [==============================] - 5519s 958ms/step - loss: 311.1496 - val_loss: 308.7574
Epoch 3/10
5759/5759 [==============================] - 5822s 1s/step - loss: 305.3264 - val_loss: 304.9336
Epoch 4/10
5759/5759 [==============================] - 6582s 1s/step - loss: 302.6262 - val_loss: 302.9958
Epoch 5/10
5759/5759 [==============================] - 7103s 1s/step - loss: 301.1792 - val_loss: 301.9337
Epoch 6/10
5759/5759 [==============================] - 14946s 3s/step - loss: 300.2849 - val_loss: 301.1922
Epoch 7/10
5759/5759 [==============================] - 4869s 846ms/step - loss: 299.6383 - val_loss: 300.6844
Epoch 8/10
5759/5759 [==============================] - 5781s 1s/step - loss: 299.1321 - val_loss: 300.2569
Epoch 9/10
5759/5759 [==============================] - 6748s 1s/step - loss: 298.7180 - val_loss: 299.8978
Epoch 10/10
5759/5759

In [46]:
hist_rec = history.history

In [47]:
hist_rec

{'loss': [332.83523499690364,
  311.1495506735721,
  305.32637336838263,
  302.6261628686316,
  301.1792370861946,
  300.28492045017515,
  299.6382695135298,
  299.1321185072753,
  298.7179960372403,
  298.3687616935474],
 'val_loss': [318.11703873779254,
  308.75738331521546,
  304.9335873032069,
  302.9957635315034,
  301.9337132302216,
  301.1921791930977,
  300.68438835990605,
  300.2568701773645,
  299.89779036501074,
  299.60179900476817],
 'testP4K_loss': [732.1974313007879,
  706.4723622909272,
  691.7300854846619,
  684.7900059335971,
  681.8280336922402,
  679.9574934180021,
  678.7282812103871,
  677.7188479585591,
  676.8431889352383,
  676.1222872974321]}

In [48]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+TfromNS+LHF_nsDELQ NN version with test loss tracking

In [68]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [69]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 128)               0   

In [70]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [71]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_09_NN7L_RH_TfromNS_LHF_nsDELQ'

In [72]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [73]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 2906s 505ms/step - loss: 202.5537 - val_loss: 189.6939
Epoch 2/20
5759/5759 [==============================] - 2024s 352ms/step - loss: 185.1673 - val_loss: 182.6220
Epoch 3/20
5759/5759 [==============================] - 1676s 291ms/step - loss: 180.0439 - val_loss: 180.6006
Epoch 4/20
5759/5759 [==============================] - 1666s 289ms/step - loss: 177.2701 - val_loss: 176.7251
Epoch 5/20
5759/5759 [==============================] - 1665s 289ms/step - loss: 175.5439 - val_loss: 175.3660
Epoch 6/20
5759/5759 [==============================] - 1634s 284ms/step - loss: 174.3043 - val_loss: 174.8005
Epoch 7/20
5759/5759 [==============================] - 1668s 290ms/step - loss: 173.3022 - val_loss: 173.3208
Epoch 8/20
5759/5759 [==============================] - 1678s 291ms/step - loss: 172.4542 - val_loss: 172.6152
Epoch 9/20
5759/5759 [==============================] - 2503s 435ms/step - loss: 171.7443 - val_loss: 172.0109
E

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+TfromNS+LHF_nsQ

In [68]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [69]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [72]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_23_MLR_RH_TfromNS_LHF_nsQ'

In [73]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [74]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [75]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 6478s 1s/step - loss: 334.0426 - val_loss: 318.8078
Epoch 2/10
5759/5759 [==============================] - 7037s 1s/step - loss: 310.9031 - val_loss: 307.7163
Epoch 3/10
5759/5759 [==============================] - 5910s 1s/step - loss: 304.0670 - val_loss: 303.4669
Epoch 4/10
5759/5759 [==============================] - 7744s 1s/step - loss: 301.0845 - val_loss: 301.4636
Epoch 5/10
5759/5759 [==============================] - 7767s 1s/step - loss: 299.5355 - val_loss: 300.3262
Epoch 6/10
5759/5759 [==============================] - 7525s 1s/step - loss: 298.5930 - val_loss: 299.6363
Epoch 7/10
5759/5759 [==============================] - 7967s 1s/step - loss: 297.9281 - val_loss: 299.1007
Epoch 8/10
5759/5759 [==============================] - 6677s 1s/step - loss: 297.4247 - val_loss: 298.6999
Epoch 9/10
5759/5759 [==============================] - 6912s 1s/step - loss: 297.0260 - val_loss: 298.3802
Epoch 10/10
5759/5759 [=====

In [76]:
hist_rec = history.history

In [77]:
hist_rec

{'loss': [334.04261498667336,
  310.90313761506275,
  304.06700017431956,
  301.08452700392866,
  299.5355056150648,
  298.59301007193477,
  297.92813114855807,
  297.42471469800324,
  297.02595976008,
  296.69932817398495],
 'val_loss': [318.8077631269736,
  307.7162972562862,
  303.4669281734807,
  301.4636359146121,
  300.3261635153653,
  299.6362964928039,
  299.10066872735194,
  298.6998974468755,
  298.3802061189176,
  298.1298349087428],
 'testP4K_loss': [717.7692014099695,
  692.5093093676612,
  681.1892812258153,
  677.2928423535407,
  675.2491797104337,
  673.9023133940009,
  672.9248305401367,
  672.2034651915096,
  671.5856063464887,
  671.1150249686433]}

In [78]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+BCONS+LHF_nsDELQ

In [95]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [96]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [97]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [98]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_23_MLR_RH_BCONS_LHF_nsDELQ'

In [99]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [100]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [101]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 8133s 1s/step - loss: 330.7609 - val_loss: 316.7712
Epoch 2/10
5759/5759 [==============================] - 6660s 1s/step - loss: 310.4906 - val_loss: 307.8904
Epoch 3/10
5759/5759 [==============================] - 7116s 1s/step - loss: 304.7100 - val_loss: 303.9924
Epoch 4/10
5759/5759 [==============================] - 7062s 1s/step - loss: 301.7960 - val_loss: 301.8254
Epoch 5/10
5759/5759 [==============================] - 8137s 1s/step - loss: 300.0964 - val_loss: 300.5073
Epoch 6/10
5759/5759 [==============================] - 7596s 1s/step - loss: 298.9904 - val_loss: 299.6046
Epoch 7/10
5759/5759 [==============================] - 7545s 1s/step - loss: 298.1876 - val_loss: 298.9532
Epoch 8/10
5759/5759 [==============================] - 7470s 1s/step - loss: 297.5743 - val_loss: 298.4452
Epoch 9/10
5759/5759 [==============================] - 7456s 1s/step - loss: 297.0949 - val_loss: 298.0184
Epoch 10/10
5759/5759 [=====

In [102]:
hist_rec = history.history

In [103]:
hist_rec

{'loss': [330.76085305019353,
  310.4906432967692,
  304.7099609745938,
  301.7960417360497,
  300.09642175858914,
  298.9904117309337,
  298.18756228044725,
  297.57428517435176,
  297.0949075244785,
  296.709771785746],
 'val_loss': [316.77116454788614,
  307.8904337055013,
  303.99240362525046,
  301.8254467632818,
  300.5073347083597,
  299.6046408204614,
  298.95319436894175,
  298.4452129081611,
  298.01840061586466,
  297.69113439902117],
 'testP4K_loss': [702.3246120228104,
  686.3533080012243,
  680.0862056377094,
  676.7398991455286,
  674.6481522557676,
  673.1816021796275,
  672.0055764137222,
  671.111931773641,
  670.3688568089174,
  669.7110506886809]}

In [104]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+NSto220+LHF_nsDELQ

In [55]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [56]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [58]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_01_MLR_RH_NSto220_LHF_nsDELQ'

In [59]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [60]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [61]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 11407s 2s/step - loss: 333.0526 - val_loss: 317.6788
Epoch 2/10
5759/5759 [==============================] - 10810s 2s/step - loss: 310.2102 - val_loss: 307.3595
Epoch 3/10
5759/5759 [==============================] - 10216s 2s/step - loss: 303.6873 - val_loss: 302.9961
Epoch 4/10
5759/5759 [==============================] - 10683s 2s/step - loss: 300.5542 - val_loss: 300.6824
Epoch 5/10
5759/5759 [==============================] - 10550s 2s/step - loss: 298.8112 - val_loss: 299.3256
Epoch 6/10
5759/5759 [==============================] - 10292s 2s/step - loss: 297.7034 - val_loss: 298.4254
Epoch 7/10
5759/5759 [==============================] - 10560s 2s/step - loss: 296.9044 - val_loss: 297.7670
Epoch 8/10
5759/5759 [==============================] - 11866s 2s/step - loss: 296.3028 - val_loss: 297.2482
Epoch 9/10
5759/5759 [==============================] - 10787s 2s/step - loss: 295.8395 - val_loss: 296.8615
Epoch 10/10
5759/57

In [62]:
hist_rec = history.history

In [63]:
hist_rec

{'loss': [333.05256303355685,
  310.2102426647167,
  303.6872879666863,
  300.55415829901773,
  298.81120031363105,
  297.7034077711415,
  296.90441165903536,
  296.30281556532356,
  295.83954845970965,
  295.4770164907051],
 'val_loss': [317.6788116925534,
  307.3594649500833,
  302.9960944944573,
  300.68239808442047,
  299.32556045328016,
  298.42535546180176,
  297.7669507749966,
  297.24817319492274,
  296.86145162218895,
  296.5717960327293],
 'testP4K_loss': [721.7310292205681,
  696.0703459624854,
  682.7236189375567,
  676.256974551415,
  672.9861661186186,
  670.9493864272823,
  669.4111307171855,
  668.1374724690269,
  667.1579007533959,
  666.4447404165944]}

In [64]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+NSto220+LHF_nsQ

In [77]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [78]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [80]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_03_MLR_RH_NSto220_LHF_nsQ'

In [81]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [82]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [83]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/10
5759/5759 [==============================] - 9714s 2s/step - loss: 333.0516 - val_loss: 317.9425
Epoch 2/10
5759/5759 [==============================] - 6439s 1s/step - loss: 310.6932 - val_loss: 308.0465
Epoch 3/10
5759/5759 [==============================] - 5183s 900ms/step - loss: 304.5171 - val_loss: 304.0613
Epoch 4/10
5759/5759 [==============================] - 6201s 1s/step - loss: 301.7091 - val_loss: 302.0904
Epoch 5/10
5759/5759 [==============================] - 6448s 1s/step - loss: 300.2337 - val_loss: 301.0068
Epoch 6/10
5759/5759 [==============================] - 6364s 1s/step - loss: 299.3400 - val_loss: 300.3276
Epoch 7/10
5759/5759 [==============================] - 5246s 911ms/step - loss: 298.7033 - val_loss: 299.8050
Epoch 8/10
5759/5759 [==============================] - 5084s 883ms/step - loss: 298.2143 - val_loss: 299.3951
Epoch 9/10
5759/5759 [==============================] - 7377s 1s/step - loss: 297.8223 - val_loss: 299.0667
Epoch 10/10
5759/57

In [84]:
hist_rec = history.history

In [85]:
hist_rec

{'loss': [333.05156581512364,
  310.6931948111193,
  304.51710277887906,
  301.70909371684877,
  300.2336531922436,
  299.34000020094226,
  298.7033277413232,
  298.2142985748487,
  297.8223111576081,
  297.49855535381016],
 'val_loss': [317.9424894715131,
  308.0464616504419,
  304.0613429893301,
  302.090381744452,
  301.0067826105784,
  300.32760883262154,
  299.8049709262554,
  299.39514361780095,
  299.06670788960923,
  298.80823058409953],
 'testP4K_loss': [722.2397246819291,
  697.4373292892026,
  684.9307260671301,
  679.3653844453859,
  676.8490013480899,
  675.4289473119096,
  674.3540836741433,
  673.5583135512015,
  672.867135846849,
  672.3943842652513]}

In [86]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Validation Check

### Chech on validation dataset (m4K simulation) to make sure the NN was properly fitted

In [ ]:
Ankitesh_models = '/oasis/scratch/comet/ankitesh/temp_project/models/'
path_aqua = 'BF.hdf5'
path_geo = 'BF_Geography.hdf5'

In [ ]:
model_aqua = load_model(Ankitesh_models+path_aqua)
model_geo = load_model(Ankitesh_models+path_geo)

tgb - 7/8/2020 - Test loading `pb` model

### TODO

In [ ]:
config_file = 'CI_SP_M4K_CONFIG.yml' # Configuration file
if path==path_aquaplanet: data_file = ['CI_SP_M4K_valid.nc','CI_SP_P4K_valid.nc']  # Validation/test data sets
elif path==path_realgeography: data_file = ['geography/CI_SP_M4K_valid.nc','geography/CI_SP_P4K_valid.nc']
NNarray = [path_aqua,path_geo]
NNname = ['Fit_aqua','Fit_geo'] # Name of NNs for plotting

In [ ]:
NN = {}; md = {};
for i,NNs in enumerate(NNarray):
    print('NN name is ',NNs)
    md[NNs] = {}
    for j,data in enumerate(data_file):
        print('data name is ',data)
        
        NN[NNs] = load_model(Ankitesh_models+NNs)
        md[NNs][data[6:-3]] = ModelDiagnostics(NN[NNs],
                                                '/home/ankitesh/CBrain_project/PrepData/'+config_file,
                                                '/oasis/scratch/comet/ankitesh/temp_project/PrepData/'+data)

In [ ]:
data_file

In [ ]:
md

In [ ]:
data_file

In [ ]:
data = data_file[1]

In [ ]:
md[NNs][data[6:-3]].valid_gen.n_batches

### Diagnostics at random drawn time steps

In [ ]:
NNarray = ['2021_01_25_O3','2021_01_25_noO3']
#dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}

In [ ]:
path = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'

In [ ]:
NN = {};
for i,NNs in enumerate(NNarray):
    print('NN name is ',NNs+'.hdf5')
    path_model = path+NNs+'.hdf5'
    NN[NNs] = load_model(path_model)

In [ ]:
gen = valid_gen_O3

$O_{3}$ vs no $O_{3}$

In [ ]:
Nt = 100
t_random = np.random.choice(np.linspace(0,1691,1692),size=((Nt,)),replace=False).astype('int')

In [ ]:
# Direct
MSE = {}
VAR = {}
INP = {}

for iar,itime in enumerate(t_random):
    print('iar=',iar,'/',Nt-1,' & itime',itime,end="\r")
    for i,NNs in enumerate(NNarray):
        if iar==0: MSE[NNs] = {}; VAR[NNs] = {}
        
        inp = gen[itime][0]
        inp_noO3 = np.delete(inp,np.arange(60,90),axis=1)
        truth = gen[itime][1]
        
        if i==0: p = NN[NNs].predict_on_batch(inp)
        elif i==1: p = NN[NNs].predict_on_batch(inp_noO3)
        
        inp_geo = np.reshape(inp,(64,128,inp.shape[1],1))
        p_geo = np.reshape(p,(64,128,p.shape[1]))
        t_geo = np.reshape(truth,(64,128,truth.shape[1]))
        
        if iar==0: 
            MSE[NNs] = np.mean((t_geo-p_geo)**2,axis=2)
            VAR[NNs] = np.var(p_geo,axis=2)
            INP[NNs] = inp_geo
        else: 
            MSE[NNs] = np.concatenate((MSE[NNs],np.mean((t_geo-p_geo)**2,axis=2)),axis=1)
            VAR[NNs] = np.concatenate((VAR[NNs],np.var(p_geo,axis=2)),axis=1)
            INP[NNs] = np.concatenate((INP[NNs],inp_geo),axis=3)

In [ ]:
INP[NNs].shape

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,89,:],axis=(1,2)),label='z=29')
plt.plot(np.mean(INP[NNs][:,:,79,:],axis=(1,2)),label='z=19')
plt.plot(np.mean(INP[NNs][:,:,69,:],axis=(1,2)),label='z=9')
plt.legend()
plt.xlabel('Latitude')
plt.ylabel('Norm O3 value')

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,89,:],axis=(0,2)),label='29')
plt.plot(np.mean(INP[NNs][:,:,79,:],axis=(0,2)),label='19')
plt.plot(np.mean(INP[NNs][:,:,69,:],axis=(0,2)),label='9')
plt.legend()
plt.xlabel('Longitude')
plt.ylabel('Norm O3 value')

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,60:90,:],axis=(0,1,3))*\
         train_gen_O3.input_transform.div[60:90]+\
         train_gen_O3.input_transform.sub[60:90])
plt.ylabel('True O3 value')
plt.xlabel('Vertical level')

In [ ]:
inp.div

In [ ]:
plt.plot(np.mean(INP[NNs][:,:,60:90,:],axis=(0,1,3)))

In [ ]:
MSE['2021_01_25_O3'].mean()

In [ ]:
MSE['2021_01_25_noO3'].mean()

In [ ]:
p.shape

In [ ]:
# Using md
MSE = {}
VAR = {}
diagno = {}
diagno['truth'] = {}
diagno['pred'] = {}

for iar,itime in enumerate(t_random):
    print('iar=',iar,'/',Nt-1,' & itime',itime,end="\r")
    for i,NNs in enumerate(NNarray):
        if iar==0: MSE[NNs] = {}; VAR[NNs] = {}

        inp, p, truth = md[NNs][data[6:-3]].get_inp_pred_truth(itime)  # [lat, lon, var, lev]
        
        t_geo = md[NNs][data[6:-3]].reshape_ngeo(truth)[:,:,:]
        p_geo = md[NNs][data[6:-3]].reshape_ngeo(p.numpy())[:,:,:]
        
        if iar==0: 
            MSE[NNs][data[6:-3]] = np.mean((t_geo-p_geo)**2,axis=2)
            VAR[NNs][data[6:-3]] = np.var(p_geo,axis=2)
        else: 
            MSE[NNs][data[6:-3]] = np.concatenate((MSE[NNs][data[6:-3]],
                                                   np.mean((t_geo-p_geo)**2,axis=2)),axis=1)
            VAR[NNs][data[6:-3]] = np.concatenate((VAR[NNs][data[6:-3]],
                                                   np.var(p_geo,axis=2)),axis=1)

#         MSE[NNs] = (MSE['samples']*MSE[NNs]+np.mean((p-truth)**2))/(MSE['samples']+1)
#         MSE['samples'] += 1

In [ ]:
for i,NNs in enumerate(NNarray):
    plt.scatter(np.mean(coor.TS,axis=(0,2)),np.log10(np.mean(MSE[NNs][data[6:-3]],axis=1)),label=NNs)
plt.legend()
plt.title(data[6:-3])

In [ ]:
for i,NNs in enumerate(NNarray):
    plt.scatter(np.mean(coor.TS,axis=(0,2)),np.mean(MSE[NNs][data[6:-3]],axis=1)/
                np.mean(VAR[NNs][data[6:-3]],axis=1),label=NNs)
plt.legend()
plt.title(data[6:-3])

In [ ]:
coor.TS.shape

In [ ]:
model_aqua.evaluate_generator(valid_gen,steps=100)

In [ ]:
model_geo.evaluate_generator(valid_gen,steps=100)

In [ ]:
model = load_model(path_HDF5+save_name+'.hdf5')
model.evaluate_generator(valid_gen)

### Check on test dataset (p4k simulation) to test ability of NN to generalize to unseen climate

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']

In [ ]:
NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
VALIDFILE = 'CI_SP_P4K_valid.nc'

valid_gen = DataGeneratorClimInv(
    data_fn = path+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)

In [ ]:
model = load_model(path_HDF5+save_name+'.hdf5')
model.evaluate_generator(valid_gen)

## Retrain BF models for transfer learning

In [ ]:
NN

In [ ]:
BF_geog = NN['BF_Geography.hdf5']

In [ ]:
BF_geog.summary()

In [ ]:
# Where to save the model
path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
save_name = 'BF_Geog_2020_07_22'

In [ ]:
#model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
BF_geog.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_pos])

## Transfer Learning experiments

tgb - 7/9/2020 - Use portion<1

### From BF aqua to BF geo

In [ ]:
TRAINFILE = 'CI_SP_M4K_train_shuffle.nc'
NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
VALIDFILE = 'CI_SP_M4K_valid.nc'

tgb - 7/23/2020 - Only training last three indices, be careful with saved names!

In [ ]:
por_array = [0.01,0.1,1]

In [ ]:
Nep = 10

In [ ]:
NN = {}
for i,por in enumerate(por_array):
    print('por=',por)
    graph = tf.Graph()
    
    #with tf.Session(graph=graph): # Legacy from tf1
        
    # 1) Define new generators
    train_gen = DataGeneratorClimInv(
    data_fn = path+TRAINFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    portion=por
    )

    valid_gen = DataGeneratorClimInv(
    data_fn = path+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    )

    # 2) Load model
    NN[por] = load_model('/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/BF_Aqua_2020_07_22.hdf5') # Load aquaplanet model

    # 3) Define callbacks and save_name of new model
    path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
    save_name = 'TL_BF_2020_07_23_porindex_'+str(i+3)
    earlyStopping = EarlyStopping(monitor='loss', patience=5, verbose=0, mode='min')
    mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='loss', mode='min')

    # 4) Train model for Nep epochs and CANNOT save state of best validation loss because
    # it would NOT be consistent with transfer learning scenario
    NN[por].fit_generator(train_gen, epochs=Nep, callbacks=[earlyStopping, mcp_save_pos])

tgb - 7/23/2020 - Legacy = Trained por_array = `[0.00001,0.0001,0.001]` already and now only training the last three indices

In [ ]:
por_array = [0.00001,0.0001,0.001,0.01,0.1,1]

In [ ]:
por_array

In [ ]:
Nep = 10

In [ ]:
NN = {}
for i,por in enumerate(por_array):
    print('por=',por)
    graph = tf.Graph()
    
    #with tf.Session(graph=graph): # Legacy from tf1
        
    # 1) Define new generators
    train_gen = DataGeneratorClimInv(
    data_fn = path+TRAINFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    portion=por
    )

    valid_gen = DataGeneratorClimInv(
    data_fn = path+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    inp_subRH=train_gen_RH.input_transform.sub, inp_divRH=train_gen_RH.input_transform.div,
    inp_subTNS=train_gen_TNS.input_transform.sub,inp_divTNS=train_gen_TNS.input_transform.div,
    rh_trans=False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    )

    # 2) Load model
    NN[por] = load_model('/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/BF_Aqua_2020_07_22.hdf5') # Load aquaplanet model

    # 3) Define callbacks and save_name of new model
    path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
    save_name = 'TL_BF_2020_07_23_porindex_'+str(i)
    earlyStopping = EarlyStopping(monitor='loss', patience=5, verbose=0, mode='min')
    mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='loss', mode='min')

    # 4) Train model for Nep epochs and CANNOT save state of best validation loss because
    # it would NOT be consistent with transfer learning scenario
    NN[por].fit_generator(train_gen, epochs=Nep, callbacks=[earlyStopping, mcp_save_pos])